<a href="https://colab.research.google.com/github/jeffblackadar/graham_fellowship/blob/master/ch_samples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Select points for quality assurance randomly.
## Objective: Select prediction points for quality assurance by a human being using a low bias manner.
For each State Game Land, select 20 points to check from bins of points separated by confidence score. Bin 1 has points of with the lowest confidence.

## Bins (5 of them): 
1. 0.70 >= and < 0.80 
2. 0.80 >= and < 0.85 
3. 0.85 >= and < 0.90 
4. 0.90 >= and < 0.95 
5. 0.95 >= 

## State game lands 217, 13, 51, 39, 83, 76, 33, 37

## Program function
For each State Game Land points are selected that fall inside its boudarie from a large list of predictions.
These points are separated into bins according to their confidence score.
4 points from each of the 5 bins are randomly selected. If a bin has four or fewer points, all of the points are selected.
A shapefile containing the points inside it is saved for each state game land. Each point has fields bin (1-5) and a bin_select (1 = randomly selected or 0 = not).

## Program inputs
* PGC_StateGamelan2018 WGS84_UTMzone18N.shp - The shapes of the PA State Game Lands
* Jeff_Predict_8_30_Ben_106_32128_buffer_count_centroid.shp - All of the points the model predicted.

## Program outputs
For each of the selected State Game Lands
* sample_predictions_binned_sgl_.shp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
!pip install geopandas

In [43]:
#https://gis.stackexchange.com/questions/92207/split-a-large-geotiff-into-smaller-regions-with-python-and-gdal
import os  
import numpy
from osgeo import gdal, osr
import math
from itertools import chain
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np
import gdalnumeric
import random



shp_output_folder = "/content/drive/My Drive/MaskCNNhearths/predictions/selections/"

sgl_polys = gpd.read_file("/content/drive/My Drive/MaskCNNhearths/Weston_Uploads/Shapefiles/PGC_StateGamelan2018 WGS84_UTMzone18N.shp")
sgl_polys = sgl_polys.to_crs(32128)
#32618
#print(sgl_polys)

#List of selected State Game Lands
#  
selected_sgls = [217, 13, 51, 39, 83, 76, 33, 37]


#Do it by steps of bins --- 4 per bin

for selected_sgls_cn in range(0,len(selected_sgls)):
    for index, row in sgl_polys.iterrows():
        sgl = row['SGL']

        if(sgl==selected_sgls[selected_sgls_cn]):
            sgl_poly = row['geometry']
            sgl_str = "000" + str(sgl)
            sgl_str = sgl_str[-3:]

    #print("sgl_poly")
    #print(sgl_poly)

    all_prediction_data = gpd.read_file("/content/drive/My Drive/MaskCNNhearths/Weston_Uploads/Shapefiles/Jeff_Predict_8_30_Ben_106_32128_buffer_count_centroid.shp")
    #all_prediction_data.to_crs()
    selection = all_prediction_data[0:]
    #print(selection)

    #Get a geodataframe to hold the points inside the poly
    points_in_sgl_poly = gpd.GeoDataFrame()
    points_in_sgl_poly['geometry'] = None
    points_in_sgl_poly.crs = 32128

    points_in_sgl_poly = gpd.GeoDataFrame(columns=["geometry","bin","bin_selected"], crs=32128)
    points_in_sgl_poly = points_in_sgl_poly.to_crs(32128)
    points_in_sgl_poly.geometry = points_in_sgl_poly.geometry.to_crs(32128)

    points_in_sgl_poly.geometry.crs = 32128
    point_count = 0
    for index, row in all_prediction_data.iterrows():
        # print("for index, row in selection.iterrows():")
        pt = row['geometry']
        #print("pt",pt)
        #print(row)
        #select only points inside the bounds of the poly
        if(pt.within(sgl_poly)==True):
            #print(row)
            point_count=point_count+1
            # print(pt, pt.within(sgl_poly))
            points_in_sgl_poly = points_in_sgl_poly.append(row, ignore_index=True)
            #print(points_in_sgl_poly)
            #print(len(points_in_sgl_poly))
    print(sgl_str, " point_count",point_count)
    
    bin_1_bottom = .70
    bin_2_bottom = .80
    bin_3_bottom = .85
    bin_4_bottom = .90
    bin_5_bottom = .95
    bin_rows = [[],[],[],[],[]]
    for index, row in points_in_sgl_poly.iterrows():
        scorenum = row['scorenum']
        if(scorenum >= bin_1_bottom and scorenum < bin_2_bottom):
            bin = 1            
        elif(scorenum >= bin_2_bottom and scorenum < bin_3_bottom):
            bin = 2
        elif(scorenum >= bin_3_bottom and scorenum < bin_4_bottom):
            bin = 3
        elif(scorenum >= bin_4_bottom and scorenum < bin_5_bottom):
            bin = 4
        elif(scorenum >= bin_5_bottom):
            bin = 5
        bin_rows[bin-1].append(index)
        #row['bin'] = bin
        points_in_sgl_poly.at[index, "bin"] = bin
        #initialize bin_selected
        points_in_sgl_poly.at[index, "bin_selected"] = 0
    #print(points_in_sgl_poly)
    #print(bin_rows)
    # For each bin, select four points
    for bin_rows_cn in range(0,5):
        #4 points less in the bin? - select them all
        if(len(bin_rows[bin_rows_cn-1]) <= 4):
            print("bin ", bin_rows_cn + 1, " has 4 or less points.")  
            for bin_points_cn in range(0,len(bin_rows[bin_rows_cn-1])):
                points_in_sgl_poly_index = bin_rows[bin_rows_cn-1][bin_points_cn]
                points_in_sgl_poly.at[points_in_sgl_poly_index, "bin_selected"] = 1
        else:
            points_selected=0  
            while(points_selected < 4):
                selected_from_bin = random.randint(0,len(bin_rows[bin_rows_cn-1])-1)
                # print("selected_from_bin",selected_from_bin,len(bin_rows[bin_rows_cn-1]))
                points_in_sgl_poly_index = bin_rows[bin_rows_cn-1][selected_from_bin]
                if(points_in_sgl_poly.at[points_in_sgl_poly_index, "bin_selected"] != 1):
                    points_in_sgl_poly.at[points_in_sgl_poly_index, "bin_selected"] = 1
                    points_selected = points_selected + 1


    outfp = os.path.join(shp_output_folder, ("sample_predictions_binned_sgl_" + sgl_str + ".shp"))
    # Write the data into that Shapefile
    points_in_sgl_poly.to_file(outfp)


217  point_count 677
013  point_count 71
051  point_count 64
039  point_count 125
083  point_count 23
bin  2  has 4 or less points.
bin  3  has 4 or less points.
bin  4  has 4 or less points.
bin  5  has 4 or less points.
076  point_count 21
bin  2  has 4 or less points.
bin  3  has 4 or less points.
bin  4  has 4 or less points.
033  point_count 56
037  point_count 22
bin  2  has 4 or less points.
bin  3  has 4 or less points.
bin  4  has 4 or less points.
